In [16]:
import scipy.io as sio

# Dataset testing

import h5py
import math
import matplotlib.pyplot as plt
import numpy as np
import scipy.fft as fft
import sigpy
import pynufft
import skimage
import os
import multiprocessing as mp
from multiprocessing import Pool
from PIL import Image
import tqdm
import glob

from training.dataset import *
from dnnlib.util import *
from torch_utils import distributed as dist
from torch_utils import misc
import training.trajectory as trajectory
import training.visualize as visualize
import generate_conditional

device = torch.device('cuda')

import torchvision

import time

import spiral_trajectory_optimization

In [17]:
a = sio.loadmat('prospective/spiral_phantom_dat.mat')['a']

In [18]:
a.shape

(942, 32, 4, 12)

In [19]:
kx = sio.loadmat('prospective/spiral_trajectory.mat')['kx']
ky = sio.loadmat('prospective/spiral_trajectory.mat')['ky']

In [20]:
kx.shape

(1, 940)

In [21]:
ky.shape

(1, 940)

In [22]:
y = np.concatenate((kx,ky), axis=0).T
y.shape

(940, 2)

In [ ]:
image = trajectory.float_to_complex(parcel[0])

y = np.array(parcel[3]['y'])[0]

points = np.array(parcel[3]['points'])

x = np.zeros((256,256))

 # Generate a fake image to use as a x_t image
blur = torchvision.transforms.GaussianBlur(kernel_size=75, sigma=2)
x_next = np.array(blur(torch.Tensor(image)), dtype = np.complex64)
nufftobj = trajectory.prealloc_nufft(x_next[0], points[0])

for i in tqdm.tqdm(range(10)):

    x_cur = x_next
    y_cur = trajectory.forward_nufft(x_cur[0], nufftobj)

    y_cur.shape
    y_grad = y-y_cur

    x_grad = trajectory.inverse_nufft(y_grad, nufftobj)

    x_next = x_cur + x_grad * 0.1

image_pil = visualize.tensor_to_image(torch.tensor(trajectory.root_summed_squares(image[0])).unsqueeze(0).unsqueeze(0), normalize=True)
x_grad_pil = visualize.tensor_to_image(torch.tensor(trajectory.root_summed_squares(x_grad)).unsqueeze(0).unsqueeze(0), normalize=True)
x_grad_next_pil = visualize.tensor_to_image(torch.tensor(trajectory.root_summed_squares(x_next)).unsqueeze(0), normalize=True)
residuals_pil = visualize.tensor_to_image(torch.tensor(trajectory.root_summed_squares((x_next-fake)**2)).unsqueeze(0), normalize=True)

visualize.image_array((image_pil, blur_pil, x_grad_next_pil, residuals_pil), columns=4)
